In [ ]:
import xarray as xr
import numpy as np
from tqdm import tqdm
import os
from geopy.distance import geodesic
from datetime import timedelta
import pickle

In [2]:
tracking_start, tracking_end = "2012-01-01", "2012-12-31"

experiment_name = 'INALT20r.L120-KRS006'#'INALT60.L120-KRS0020'
data_resolution = '1d'

OW_thr_factor =-0.3

In [3]:
if experiment_name.startswith("INALT60"):
    prefix = "2_"
    Npix_min, Npix_max = 720, 18000
    sigma = 15
    wx = 600
elif experiment_name.startswith("INALT20"):
    prefix = "1_"
    Npix_min, Npix_max = 80, 2000
    sigma = 5
    wx = 200

In [4]:
#depth= 0  #corresponding to... 
#depth_index = 0 

mesh_mask = xr.open_dataset(f'/gxfs_work/geomar/smomw523/smoothed_data/{experiment_name}/{prefix}{experiment_name}_mesh_mask.nc') 
indices = np.concatenate((range(0, 11, 10),range(18, 25, 6),range(29, 34, 4),range(36, 40, 3),range(41, 120, 2)))
depth_information = [(round(mesh_mask.nav_lev.values[i]), i) for i in indices]
print(len(depth_information),depth_information)

48 [(0, 0), (21, 10), (54, 18), (91, 24), (132, 29), (173, 33), (210, 36), (253, 39), (286, 41), (323, 43), (365, 45), (412, 47), (465, 49), (525, 51), (592, 53), (668, 55), (753, 57), (848, 59), (952, 61), (1066, 63), (1189, 65), (1321, 67), (1461, 69), (1608, 71), (1762, 73), (1922, 75), (2086, 77), (2255, 79), (2428, 81), (2603, 83), (2782, 85), (2963, 87), (3146, 89), (3331, 91), (3518, 93), (3706, 95), (3896, 97), (4086, 99), (4279, 101), (4472, 103), (4666, 105), (4861, 107), (5057, 109), (5254, 111), (5452, 113), (5651, 115), (5850, 117), (6050, 119)]


In [ ]:
#min_diameter, min_duration = 50, 50
filename = f'Tracks-crossed_{tracking_start.replace("-", "")}_{tracking_end.replace("-", "")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.pickle'

for (depth, depth_index) in tqdm(depth_information[:2]): # depth_information
    datapath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'
    filepath = datapath + filename
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        continue

    tracks = np.load(datapath + filename, allow_pickle=True)
    for track in tracks:
        #if track['scale'].max() < min_diameter or len(track['time']) < min_duration:
        #    continue
        #else:
        speeds = [0]
        times, lons, lats = track['time'], track['lon'], track['lat']
        for i in range(1, len(times)):
            dist_km = geodesic((lats[i-1], lons[i-1]), (lats[i], lons[i])).km
            time_diff_h = (times[i] - times[i-1]) / np.timedelta64(1, 'h')
            speed = dist_km / time_diff_h * 3.6
            speeds.append(speed)
        track['speed'] = np.array(speeds)
    with open(filepath, "wb") as f:
        pickle.dump(tracks, f)

100%|██████████| 48/48 [01:00<00:00,  1.26s/it]
